REQUIRED INSTALLS:

import sys
!{sys.executable} -m pip install tensorflow==2.0.0-rc1 --user

import sys
!{sys.executable} -m pip install keras

REQUIRED IMPORTS:

* import numpy as np
* import tensorflow as tf
* from keras.models import Sequential
* from keras.layers import Convolution2D
* from keras.layers import MaxPooling2D
* from keras.layers import Flatten
* from keras.layers import Dense
* from keras.preprocessing.image import ImageDataGenerator
* from keras.preprocessing import image
* from IPython.display import display
* from PIL import Image
* import os

In [2]:
def getFilesFromDir(path, extension): #ext should include '.', such as '.jpg'
    import os
    fileList = []
    for root, dirs, files in os.walk(path):
        for filename in files:
            if filename[-len(extension):] == extension: # gets the extension from filename and compares it
                fileList.append(filename)
    return fileList

In [3]:
def performTrainingDataAugmentation(): #Re-visit!!!
    from keras.preprocessing.image import ImageDataGenerator
    return ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

def performTestingDataAugmentation(): #Re-visit!!!
    from keras.preprocessing.image import ImageDataGenerator
    return ImageDataGenerator(rescale = 1./255)  

In [4]:
def setup_TrainingSet(path):
    train_datagen = performTrainingDataAugmentation()
    return train_datagen.flow_from_directory(
    path,
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary')
    
def setup_TestingSet(path):
    test_datagen = performTestingDataAugmentation()
    return test_datagen.flow_from_directory(
    path,
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary')


In [5]:
def setup_Classifier(classifier_type, activ, metricsArray):
    import tensorflow as tf
    from keras.models import Sequential
    from keras.layers import Convolution2D
    from keras.layers import MaxPooling2D
    from keras.layers import Flatten
    from keras.layers import Dense
    clasf = Sequential()    # Step 0: Create the classifier
    clasf.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu')) # Step 1: Convolution: extract features from image input
    clasf.add(MaxPooling2D(pool_size = (2, 2))) # Step 2: Pooling: dimentionality reduction
    clasf.add(Flatten()) # Step 3: Flattening: dimentionally-reduced matrix is converted to linear array for single-element outputing
    clasf.add(Dense(activation = 'relu', units = 128))
    clasf.add(Dense(activation = 'sigmoid', units = 1))
    clasf.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return clasf

In [27]:
def predictions(classifier, path, extension, outputA, outputB):
    import numpy as np
    from keras.preprocessing import image
    predList = []
    for filename in getFilesFromDir(path, extension):
        test_image = image.load_img((path + '/' + filename), target_size = (64, 64)) #concatenates Gwann/img and random.jpg together
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = classifier.predict(test_image)
        training_set.class_indices
        if result[0][0] >= 0.5:
            predList.append((outputA, result[0][0]))
        else:
            predList.append((outputB, result[0][0]))
    return predList

def results(classifier, path, extension, outputA, outputB):
    predList = predictions(classifier, path, extension, outputA, outputB)
    correct = 0
    for predTuple in predList: # predTuple = (output, confidence)
        print("conf: " + str(predTuple[1]))
        if predTuple[0] == outputA:
            print("Believed to be: " + outputA + ". Confidence: " + str(predTuple[1]))
            correct += 1
        else:
            print("Believed to be: " + outputB + ". Confidence: " + str(predTuple[1]))
    print("Accuracy: " + str(round((100.0 * (correct / len(predList))), 3)) + '%')

In [28]:
def trainModel(classif_type, activ, metricsArr, training_set, numEpochs, stepsPerEpoch, testing_set, validationSteps):
    from IPython.display import display
    from PIL import Image
    clasf = setup_Classifier(classif_type, activ, metricsArr)
    clasf.fit_generator(
        training_set,
        steps_per_epoch = stepsPerEpoch,
        epochs = numEpochs,
        validation_data = testing_set,
        validation_steps = validationSteps
    )
    return clasf

_Running through Hot-Dog / Not Hot-Dog example with wrapper_:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

training_set = setup_TrainingSet('Gwann/seefood/train')
testing_set = setup_TestingSet('Gwann/seefood/test') # ResNet50
model = trainModel('Sequential', 'relu', ['accuracy'], training_set, 4, 30, testing_set, 20)
results(model, 'Gwann/seefood/test/hot_dog', '.jpg', 'Hot Dog', 'Not Hot Dog')

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Epoch 1/4
30/30 [==============================] - 7s 237ms/step - loss: 0.9563 - accuracy: 0.5000 - val_loss: 0.7350 - val_accuracy: 0.5398
Epoch 2/4
30/30 [==============================] - 7s 243ms/step - loss: 0.6180 - accuracy: 0.6524 - val_loss: 0.6925 - val_accuracy: 0.5669
Epoch 3/4
15/30 [==============>...............] - ETA: 2s - loss: 0.5957 - accuracy: 0.6953